In [1]:
import os

from pandas import DataFrame, date_range
import numpy as np

from idealreport import create_html
from idealreport.reporter import Reporter
import htmltag

In [2]:
# instantiate reporter object and set output location

output_path = '/home/jason/ideal/reports/sample_report/output'
output_file = os.path.join(output_path, 'sample_plots.html')
r = Reporter(title='Sample Plots', output_file=output_file)

# report start
r.h += htmltag.h3('This report provides an overivew of the different plot types.')

In [3]:
# vertical bar chart

ch = DataFrame({'Entity': ['Entity 1', 'Entity 2', 'Entity 3', 'Entity 4', 'Entity 5'],
                  'Stat 1': np.random.randn(5).tolist(),
                  'Stat 2': np.random.randn(5).tolist(),
                })

# note the x values are generally assumed to be in the index, with each non index column being a series of y values
# a few exceptions are outlined below
ch = ch.set_index('Entity')
ch = ch.sort_values(by='Stat 1', ascending=False)

# each object can be built by create_html for more flexibility

# graphs are generated from dictionaries with attributes and a data dictionary as an entry
bar_plot = {
    'title': 'Vertical Bar Chart by Create HTML',
    'data': [
        {
            'df': ch,
            'type': 'bar',
            'orientation': 'v',
        },
    ],
    'labelX': 'Entity',
    'labelY': '%',
    'staticPlot': False
}

r.h += htmltag.h4('Bar Charts')
r.h += create_html.plot(bar_plot)

# all of these examples just use the reporter object to build the dictionaries for create_html

r.plot.bar(df=ch, title='Vertical Bar Chart by Reporter', xlabel='Entity', ylabel='%')

{'data': [{'df':             Stat 1    Stat 2
   Entity                      
   Entity 2  0.299653  1.211036
   Entity 1  0.091964 -0.007555
   Entity 5 -0.307883  1.251296
   Entity 4 -0.461797  1.524542
   Entity 3 -0.510706 -0.638570, 'orientation': 'v', 'type': 'bar'}],
 'staticPlot': False,
 'title': 'Vertical Bar Chart by Reporter',
 'x': {'label': 'Entity'},
 'y': {'label': '%'}}

In [4]:
# stacked horizontal bar chart, notice additional entry in the data field for orientation

ch = DataFrame({'Entity': ['Entity 1', 'Entity 2', 'Entity 3', 'Entity 4', 'Entity 5'],
                'Foo': np.random.randn(5).tolist(),
                'Bar': np.random.randn(5).tolist(),
               })
ch = ch.set_index('Entity')
ch = ch[['Foo', 'Bar']] # make sure columns have desired order

# stacked bars behave oddly with mixed positive / negative numbers, so only use when all numbers are positive or negative within a series
ch = ch.abs()
ch = ch*100

r.plot.barh(df=ch, title='Horizontal Stacked Bar Chart', stacked=True, xlabel='$')

{'data': [{'df':                  Foo         Bar
   Entity                          
   Entity 1  165.063188  159.536089
   Entity 2  133.794515   64.874633
   Entity 3   33.105849   58.210632
   Entity 4   95.010840   13.240251
   Entity 5  193.778160  204.927478,
   'orientation': 'h',
   'type': 'stackedBar'}],
 'staticPlot': False,
 'title': 'Horizontal Stacked Bar Chart',
 'x': {'label': '$'}}

In [5]:
# line chart

ch = DataFrame(np.random.randn(20, 3))
ch.columns = ['a', 'b', 'c']
ch.a = range(20)
ch = ch.set_index('a')

r.h += htmltag.h4('Line Charts')
r.plot.line(df=ch, title='Line Chart', xlabel='Entity', ylabel='%')

{'data': [{'df':            b         c
   a                     
   0   0.514645 -1.710508
   1   1.530271 -1.077341
   2  -0.920036 -0.718898
   3  -0.269135 -0.304892
   4  -0.869412 -0.654376
   5   0.526337  0.081248
   6   0.226730  0.711125
   7  -0.804674  0.131844
   8   0.236074  1.482854
   9   0.288534 -0.127145
   10  0.606228 -0.699456
   11  0.751652  0.996776
   12  0.790983 -0.251756
   13 -1.362174  0.193250
   14  2.105469 -0.444325
   15  0.555961 -0.767429
   16  0.986980  0.529327
   17 -0.386982 -0.752673
   18  1.105594  0.374998
   19 -0.829967 -0.557065, 'type': 'line'}],
 'staticPlot': False,
 'title': 'Line Chart',
 'x': {'label': 'Entity'},
 'y': {'label': '%'}}

In [6]:
# time series

ind = date_range('2017-11-02 9:00', periods=20, freq='T')
ch = DataFrame(data=np.random.randn(20,2), index=ind)
ch.columns = ['a', 'b']

# just like a regular line except the index is timestamps
r.plot.time(df=ch, title='Time Series', xlabel='Time', ylabel='%')

{'data': [{'df':                             a         b
   2017-11-02 09:00:00  0.164488  1.418754
   2017-11-02 09:01:00  1.735531  0.232345
   2017-11-02 09:02:00  0.695366 -1.092543
   2017-11-02 09:03:00 -0.915197  0.020179
   2017-11-02 09:04:00 -1.359061 -0.284135
   2017-11-02 09:05:00  2.305699 -0.036743
   2017-11-02 09:06:00 -2.222610 -0.630516
   2017-11-02 09:07:00  1.809981 -1.047924
   2017-11-02 09:08:00 -1.343169  0.580274
   2017-11-02 09:09:00 -0.002732 -0.819442
   2017-11-02 09:10:00  1.618822 -0.537718
   2017-11-02 09:11:00 -0.863856 -0.044852
   2017-11-02 09:12:00  1.020124  1.183619
   2017-11-02 09:13:00  0.305703  0.113012
   2017-11-02 09:14:00  0.162127  0.666935
   2017-11-02 09:15:00 -0.067257 -0.357314
   2017-11-02 09:16:00 -1.234905  0.542565
   2017-11-02 09:17:00  0.752043 -0.810283
   2017-11-02 09:18:00 -1.872425  0.750384
   2017-11-02 09:19:00 -0.180755 -1.406872, 'type': 'line'}],
 'staticPlot': False,
 'title': 'Time Series',
 'x': {'label': '

In [7]:
# scatter plot

ch = DataFrame(np.random.randn(20, 3))
ch.columns = ['a', 'b', 'c']
ch = ch.set_index('a')

r.h += htmltag.h4('Scatter Plots')
r.plot.scatter(df=ch, title='Scatter Plot', xlabel='alpha', ylabel='beta')

{'data': [{'df':                   b         c
   a                            
    0.483340 -0.089462 -0.294429
    0.753940  0.576042  1.336779
   -0.616452 -0.646941  0.920131
    1.242979 -1.270761  0.657224
    0.650529  0.652512 -0.234936
    0.868040 -0.191551  3.570574
   -0.174106  2.072705 -0.021228
    1.961306  0.615299  0.220797
    0.306025  0.713298 -0.053590
   -0.299655  1.553355 -0.969138
    0.671217  0.407553  0.702356
    0.212278 -1.605571  0.684166
    0.932841  1.490191 -1.056475
   -0.509551  0.656310 -0.665621
   -0.593829  1.317131  0.872824
   -0.917337  1.231286 -0.081187
    1.274666 -0.942772  0.542424
   -1.090429 -1.249282 -0.476124
   -0.642836 -0.231424 -0.205839
   -1.783400 -1.375024  2.123331, 'type': 'scatter'}],
 'staticPlot': False,
 'title': 'Scatter Plot',
 'x': {'label': 'alpha'},
 'y': {'label': 'beta'}}

In [8]:
# mixed plot types on one graph

ch1 = DataFrame(np.random.randn(20, 2))
ch1.columns = ['a', 'b1']
ch1.sort_values(by='a', ascending=True, inplace=True)
ch1 = ch1.set_index('a')

ch2 = DataFrame(np.random.randn(20, 2))
ch2.columns = ['a', 'b2']
ch2.sort_values(by='a', ascending=True, inplace=True)
ch2 = ch2.set_index('a')

r.h += htmltag.h4('Mixed Type Plots')
r.plot.multi(dfs=[ch1, ch2], types=['line', 'bar'], title='Mixed Line and Bar Plot', xlabel='x label', ylabel='y label')

{'data': [{'df':                  b1
   a                  
   -1.626203 -1.297141
   -1.277642 -0.825630
   -1.275274  0.196175
   -1.164451 -0.138213
   -1.119178 -0.089327
   -0.696649 -1.442830
   -0.677210 -0.289843
   -0.568898  0.206030
   -0.451980  0.345014
   -0.445884  0.241573
   -0.433120 -0.129361
    0.201412 -0.092565
    0.316012 -0.030878
    0.327010 -0.260897
    0.343924 -3.054516
    0.508455  0.999153
    0.589792 -1.837594
    1.083997 -1.762111
    1.315986 -1.939990
    1.356370  0.970772, 'type': 'line'}, {'df':                  b2
   a                  
   -1.973560  0.483220
   -1.783090  1.726250
   -1.531924  0.531902
   -1.246060 -1.063610
   -1.062432  1.511462
   -0.996776 -0.116299
   -0.786064 -1.130753
   -0.720978 -0.579146
   -0.699593 -0.532944
   -0.544406  0.955205
   -0.350022 -0.422521
   -0.294387  0.889307
   -0.269425 -0.112062
   -0.115948  0.445866
    0.523485  0.915453
    0.530229  0.538794
    1.153809  1.221293
    1.283139  2.02830

In [9]:
# different axes

ch1 = DataFrame(np.random.randn(20, 2))
ch1.columns = ['x', 'a']
ch1.x = range(20)
ch1 = ch1.set_index('x')

ch2 = DataFrame(np.random.randn(20, 2))
ch2.columns = ['x', 'b']
ch2.x = range(20)
ch2 = ch2.set_index('x')
ch2.b *= 100

# y2_axis is a list of booleans indicating whether each df should be tied to the y2 axis or not
r.plot.multi(dfs=[ch1, ch2], types=['line', 'line'], title='Lines with Different Y Axes', xlabel='x', ylabel='ya', y2_axis=[False,True], y2label='yb')

{'data': [{'df':            a
   x           
   0   0.485286
   1  -1.490647
   2   0.000469
   3   1.041183
   4   1.187626
   5   1.219242
   6   0.824595
   7  -1.280597
   8   0.454238
   9  -0.463662
   10 -0.794674
   11 -1.345128
   12 -0.818671
   13  0.544182
   14  0.411447
   15  0.295879
   16  0.775119
   17  1.048946
   18 -0.788249
   19  0.206882, 'type': 'line', 'y2': False}, {'df':              b
   x             
   0    82.883191
   1    51.718255
   2   135.244039
   3  -136.541759
   4   -33.889934
   5   -29.536333
   6  -154.932228
   7   163.927374
   8   171.849169
   9  -131.573455
   10  -74.450200
   11   55.879422
   12   22.136987
   13   83.718975
   14   -5.984934
   15   71.428061
   16  -22.373818
   17  -34.899033
   18   13.543579
   19   74.515732, 'type': 'line', 'y2': True}],
 'staticPlot': False,
 'title': 'Lines with Different Y Axes',
 'x': {'label': 'x'},
 'y': {'label': 'ya'},
 'y2': {'label': 'yb'}}

In [10]:
# ohlc

ch = DataFrame(np.random.randn(4, 4))
ch.columns = ['open', 'high', 'low', 'close']
ch['high'] = ch['high'] + 3
ch['low'] = ch['low'] - 3

ch['dealer'] = range(4)
ch = ch.set_index('dealer')

r.h += htmltag.h4('Open High Low Close (OHCL) Plot')
r.plot.ohlc(df=ch, title='OHLC Plot by Reporter', xlabel='price', ylabel='instrument')

{'data': [{'df':             open      high       low     close
   dealer                                        
   0       0.260311  3.010310 -2.672450  0.738853
   1      -0.497054  3.773390 -2.924505  0.156617
   2       0.684079  4.722876 -3.823871  1.418983
   3       0.163511  2.308643 -2.451459  0.081337, 'type': 'ohlc'}],
 'name': '',
 'staticPlot': False,
 'title': 'OHLC Plot by Reporter',
 'x': {'label': 'price'},
 'y': {'label': 'instrument'}}

In [11]:
# generate and save the report HTML
r.generate()

saved report to /home/jason/ideal/reports/sample_report/output/sample_plots.html
